In [1]:
import numpy as np
import pandas as pd


In [2]:
df = pd.read_csv("/content/diabetes.csv")

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.shape

(768, 9)

In [5]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [6]:
x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [8]:
X = sc.fit_transform(x)

In [9]:
X

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [11]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [12]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [13]:
model.fit(X_train,y_train,batch_size=32,epochs=100, validation_data=(X_test,y_test))

Epoch 1/100
20/20 [==============================] - 2s 13ms/step - loss: 0.6643 - accuracy: 0.6303 - val_loss: 0.6060 - val_accuracy: 0.7143
Epoch 2/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6179 - accuracy: 0.6775 - val_loss: 0.5718 - val_accuracy: 0.7403
Epoch 3/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5843 - accuracy: 0.7199 - val_loss: 0.5441 - val_accuracy: 0.7597
Epoch 4/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5591 - accuracy: 0.7410 - val_loss: 0.5232 - val_accuracy: 0.7468
Epoch 5/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5407 - accuracy: 0.7622 - val_loss: 0.5094 - val_accuracy: 0.7532
Epoch 6/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5252 - accuracy: 0.7573 - val_loss: 0.4965 - val_accuracy: 0.7597
Epoch 7/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5126 - accuracy: 0.7541 - val_loss: 0.4869 - val_accuracy: 0.7597
Epoch 8/100


In [14]:
pip install -U keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
import keras_tuner as kt

In [16]:
def build_model(hp):

  model = Sequential()
  model.add(Dense(32,activation='relu',input_dim = 8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer = hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adadelta'])

  model.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [17]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials=5)

In [18]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [19]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [20]:
model = tuner.get_best_models(num_models=1)[0]

In [21]:
model.fit(X_train,y_train, batch_size=32,epochs = 100,initial_epoch=6,validation_data=(X_test, y_test))

Epoch 7/100
20/20 [==============================] - 1s 15ms/step - loss: 0.5277 - accuracy: 0.7573 - val_loss: 0.5212 - val_accuracy: 0.7727
Epoch 8/100
20/20 [==============================] - 0s 8ms/step - loss: 0.5118 - accuracy: 0.7622 - val_loss: 0.5087 - val_accuracy: 0.7792
Epoch 9/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4993 - accuracy: 0.7720 - val_loss: 0.4990 - val_accuracy: 0.7727
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4897 - accuracy: 0.7736 - val_loss: 0.4916 - val_accuracy: 0.7727
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4821 - accuracy: 0.7769 - val_loss: 0.4862 - val_accuracy: 0.7727
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4763 - accuracy: 0.7720 - val_loss: 0.4810 - val_accuracy: 0.7662
Epoch 13/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4711 - accuracy: 0.7752 - val_loss: 0.4767 - val_accuracy: 0.7662
Epoch 14

In [22]:
def build_model(hp):
  model = Sequential()

  units = hp.Int('units',min_value = 8, max_value = 128,step=8)

  model.add(Dense(units = units, activation = 'relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer = 'rmsprop',loss = 'binary_crossentropy', metrics=['accuracy'])

  return model

In [23]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                       max_trials=5,
                        directory='mydir',
                        project_name = 'my_project')

In [24]:
tuner.search(X_train,y_train,epochs = 5,validation_data=(X_test,y_test))

In [25]:
tuner.get_best_hyperparameters()[0].values

{'units': 112}

In [26]:
model = tuner.get_best_models(num_models=1)[0]

In [27]:
model.fit(X_train,y_train,epochs=100, initial_epoch=6, validation_data=(X_train,y_train))

Epoch 7/100
20/20 [==============================] - 2s 22ms/step - loss: 0.4965 - accuracy: 0.7720 - val_loss: 0.4838 - val_accuracy: 0.7752
Epoch 8/100
20/20 [==============================] - 0s 16ms/step - loss: 0.4814 - accuracy: 0.7769 - val_loss: 0.4718 - val_accuracy: 0.7834
Epoch 9/100
20/20 [==============================] - 0s 11ms/step - loss: 0.4713 - accuracy: 0.7834 - val_loss: 0.4636 - val_accuracy: 0.7899
Epoch 10/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4648 - accuracy: 0.7850 - val_loss: 0.4578 - val_accuracy: 0.7899
Epoch 11/100
20/20 [==============================] - 0s 10ms/step - loss: 0.4593 - accuracy: 0.7883 - val_loss: 0.4537 - val_accuracy: 0.7915
Epoch 12/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4556 - accuracy: 0.7866 - val_loss: 0.4505 - val_accuracy: 0.7915
Epoch 13/100
20/20 [==============================] - 0s 10ms/step - loss: 0.4526 - accuracy: 0.7915 - val_loss: 0.4479 - val_accuracy: 0.7899
Epoc

In [28]:
def build_model(hp):

  model = Sequential()

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    model.add(Dense(72,activation='relu'))
  
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [29]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 3,
                        directory = 'mydir',
                        project_name = 'num_layer')

In [30]:
tuner.search(X_train,y_train,epochs=10,validation_data=(X_test,y_test))

In [31]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 5}

In [32]:
model = tuner.get_best_models(num_models=1)[0]

In [33]:
model.fit(X_train,y_train,epochs=100, initial_epoch=6, validation_data=(X_train,y_train))

Epoch 7/100
20/20 [==============================] - 3s 15ms/step - loss: 0.3934 - accuracy: 0.8176 - val_loss: 0.4484 - val_accuracy: 0.7769
Epoch 8/100
20/20 [==============================] - 0s 9ms/step - loss: 0.3876 - accuracy: 0.8127 - val_loss: 0.3717 - val_accuracy: 0.8306
Epoch 9/100
20/20 [==============================] - 0s 8ms/step - loss: 0.3755 - accuracy: 0.8371 - val_loss: 0.3504 - val_accuracy: 0.8388
Epoch 10/100
20/20 [==============================] - 0s 12ms/step - loss: 0.3694 - accuracy: 0.8322 - val_loss: 0.3607 - val_accuracy: 0.8306
Epoch 11/100
20/20 [==============================] - 0s 12ms/step - loss: 0.3582 - accuracy: 0.8453 - val_loss: 0.3374 - val_accuracy: 0.8453
Epoch 12/100
20/20 [==============================] - 0s 10ms/step - loss: 0.3528 - accuracy: 0.8371 - val_loss: 0.3224 - val_accuracy: 0.8583
Epoch 13/100
20/20 [==============================] - 0s 15ms/step - loss: 0.3440 - accuracy: 0.8404 - val_loss: 0.3066 - val_accuracy: 0.8795
Epoc

In [34]:
from tensorflow.python.distribute.input_lib import values
def build_model(hp):
  model = Sequential()

  counter = 0

  for i in range(hp.Int('num_layers', min_value = 1, max_value=10)):
    if counter == 0:

      model.add(Dense(hp.Int('units'+ str(i), min_value = 8,max_value = 128,step = 8),activation = hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']),input_dim = 8))
      model.add(Dropout(hp.Choice('dropout'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
      model.add(Dense(hp.Int('units'+str(i), min_value = 8,max_value = 128,step = 8),activation = hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid'])))
      model.add(Dropout(hp.Choice('dropout'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    counter+=1

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adadelta']),
                    loss = 'binary_crossentropy',
                    metrics = ['accuracy'])
  return model
        


In [35]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        directory = 'mydir',
                        project_name = 'final1')

In [36]:
tuner.search(X_train,y_train, epochs = 5, validation_data=(X_test,y_test))

Trial 10 Complete [00h 00m 03s]
val_accuracy: 0.30519479513168335

Best val_accuracy So Far: 0.7662337422370911
Total elapsed time: 00h 00m 43s


In [37]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4,
 'units0': 16,
 'activation0': 'tanh',
 'dropout0': 0.2,
 'optimizer': 'adam',
 'units1': 24,
 'activation1': 'tanh',
 'dropout1': 0.1,
 'units2': 8,
 'activation2': 'tanh',
 'dropout2': 0.1,
 'units3': 112,
 'activation3': 'sigmoid',
 'dropout3': 0.2,
 'units4': 40,
 'activation4': 'relu',
 'dropout4': 0.3,
 'units5': 80,
 'activation5': 'tanh',
 'dropout5': 0.5,
 'units6': 32,
 'activation6': 'relu',
 'dropout6': 0.7,
 'units7': 8,
 'activation7': 'relu',
 'dropout7': 0.1,
 'units8': 120,
 'activation8': 'relu',
 'dropout8': 0.5,
 'units9': 56,
 'activation9': 'relu',
 'dropout9': 0.9}

In [38]:
model = tuner.get_best_models(num_models=1)[0]

In [39]:
model.fit(X_train,y_train,epochs = 100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 2s 20ms/step - loss: 0.5579 - accuracy: 0.7182 - val_loss: 0.4960 - val_accuracy: 0.7532
Epoch 8/100
20/20 [==============================] - 0s 10ms/step - loss: 0.5293 - accuracy: 0.7345 - val_loss: 0.4908 - val_accuracy: 0.7338
Epoch 9/100
20/20 [==============================] - 0s 9ms/step - loss: 0.5023 - accuracy: 0.7687 - val_loss: 0.4800 - val_accuracy: 0.7597
Epoch 10/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4970 - accuracy: 0.7606 - val_loss: 0.4842 - val_accuracy: 0.7597
Epoch 11/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4841 - accuracy: 0.7785 - val_loss: 0.4823 - val_accuracy: 0.7662
Epoch 12/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4735 - accuracy: 0.7638 - val_loss: 0.4866 - val_accuracy: 0.7532
Epoch 13/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4702 - accuracy: 0.7736 - val_loss: 0.4811 - val_accuracy: 0.7532
Epoch 1